In [1]:
import tensorflow as tf
import os
import time
from get_mnist import read_data_sets
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
mnist = read_data_sets("./mnist", one_hot=True, dtype='float32')

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [3]:
with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, [None, 784], name="input")
    y = tf.placeholder(tf.float32, [None, 10], name="output")

# Define network

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


In [5]:
W_conv1 = weight_variable([10, 10, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

output=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    
W1_image = tf.transpose(W_conv1, [3,0,1,2])
tf.image_summary("W_conv1", W1_image, max_images=32)

# with tf.name_scope("loss"):
cross_entropy = -tf.reduce_sum(y*tf.log(output + 1e-9))
xentropy = tf.scalar_summary("xentropy", cross_entropy)
    
# with tf.name_scope("train"):
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# with tf.name_scope("test"):
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_summary = tf.scalar_summary("accuracy", accuracy)

In [ ]:
merged = tf.merge_all_summaries()
timestring = datetime.datetime.now().strftime("mnist_conv_%Y%m%d-%H%M%S")
with tf.Session() as sess:
    writer = tf.train.SummaryWriter("./tensorboard/{}/".format(timestring), sess.graph_def)
    sess.run(tf.initialize_all_variables())
    for i in range(10000):
        x_tr_, y_tr_ = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: x_tr_, y: y_tr_, keep_prob:0.5})
        if i % 10 == 0:
            x_te_, y_te_ = mnist.validation.next_batch(100)
            print "{}% correct".format(sess.run(100*accuracy, feed_dict={x: x_te_, y: y_te_, keep_prob:1.0}))
            summaries = sess.run(merged, feed_dict={x: x_te_, y: y_te_, keep_prob:1.0})
            writer.add_summary(summaries, i)

14.0% correct
22.0% correct
52.9999961853% correct
74.0% correct
76.0% correct
77.0% correct
73.0% correct
81.0% correct
82.0% correct
85.0% correct
83.0% correct
87.0% correct
87.0% correct
90.0% correct
94.0% correct
94.0% correct
92.0% correct
97.0% correct
92.0% correct
92.0% correct
91.0% correct
93.0% correct
90.0% correct
94.0% correct
92.0% correct
95.0% correct